If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [1]:
#! pip install datasets transformers

If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [2]:
from huggingface_hub import notebook_login

notebook_login()

Then you need to install Git-LFS. Uncomment the following instructions:

In [3]:
# !apt install git-lfs

Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [4]:
import transformers

print(transformers.__version__)

4.26.1


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/language-modeling).

We also quickly upload some telemetry - this tells us which examples and software versions are getting used so we know where to prioritize our maintenance efforts. We don't collect (or care about) any personally identifiable information, but if you'd prefer not to be counted, feel free to skip this step or delete this cell entirely.

In [5]:
from transformers.utils import send_example_telemetry

send_example_telemetry("language_modeling_notebook", framework="pytorch")

# Fine-tuning a language model

In this notebook, we'll see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model on a language modeling tasks. We will cover two types of language modeling tasks which are:

- Causal language modeling: the model has to predict the next token in the sentence (so the labels are the same as the inputs shifted to the right). To make sure the model does not cheat, it gets an attention mask that will prevent it to access the tokens after token i when trying to predict the token i+1 in the sentence.

![Widget inference representing the causal language modeling task](images/causal_language_modeling.png)

- Masked language modeling: the model has to predict some tokens that are masked in the input. It still has access to the whole sentence, so it can use the tokens before and after the tokens masked to predict their value.

![Widget inference representing the masked language modeling task](images/masked_language_modeling.png)

We will see how to easily load and preprocess the dataset for each one of those tasks, and how to use the `Trainer` API to fine-tune a model on it.

A script version of this notebook you can directly run on a distributed environment or on TPU is available in our [examples folder](https://github.com/huggingface/transformers/tree/master/examples).

## Preparing the dataset

For each of those tasks, we will use the [Wikitext 2]() dataset as an example. You can load it very easily with the 🤗 Datasets library.

In [6]:
from datasets import load_dataset
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')

Found cached dataset wikitext (C:/Users/Admin/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
100%|██████████| 3/3 [00:00<00:00, 750.19it/s]


You can replace the dataset above with any dataset hosted on [the hub](https://huggingface.co/datasets) or use your own files. Just uncomment the following cell and replace the paths with values that will lead to your files:

In [7]:
# datasets = load_dataset("text", data_files={"train": path_to_train.txt, "validation": path_to_validation.txt}

You can also load datasets from a csv or a JSON file, see the [full documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) for more information.

To access an actual element, you need to select a split first, then give an index:

In [8]:
datasets["train"][10]

{'text': ' The game \'s battle system , the BliTZ system , is carried over directly from Valkyira Chronicles . During missions , players select each unit using a top @-@ down perspective of the battlefield map : once a character is selected , the player moves the character around the battlefield in third @-@ person . A character can only act once per @-@ turn , but characters can be granted multiple turns at the expense of other characters \' turns . Each character has a field and distance of movement limited by their Action Gauge . Up to nine characters can be assigned to a single mission . During gameplay , characters will call out if something happens to them , such as their health points ( HP ) getting low or being knocked out by enemy attacks . Each character has specific " Potentials " , skills unique to each character . They are divided into " Personal Potential " , which are innate skills that remain unaltered unless otherwise dictated by the story and can either help or impede

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [9]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [10]:
show_random_elements(datasets["train"])

,text
0,"198th & 234th Batteries , 73rd Anti @-@ Tank Regiment , RA \n"
1,"The US military acknowledged the non @-@ governmental organizations ' complaints concerning flight @-@ operations bias and promised improvement while noting that up to 17 January 600 emergency flights had landed and 50 were diverted ; by the first weekend of disaster operations diversions had been reduced to three on Saturday and two on Sunday . The airport was able to support 100 landings a day , up from the 35 a day that the airport gets during normal operation . A spokesman for the joint task force running the airport confirmed that though more flights were requesting landing slots , none were being turned away . \n"
2,"Ireland once dominated the world 's market for whiskey , producing 90 % of the world 's whiskey at the start of the 20th century . However , as a consequence of bootleggers during the prohibition in the United States ( who sold poor @-@ quality whiskey bearing Irish @-@ sounding names thus eroding the pre @-@ prohibition popularity for Irish brands ) and tariffs on Irish whiskey across the British Empire during the Anglo @-@ Irish Trade War of the 1930s , sales of Irish whiskey worldwide fell to a mere 2 % by the mid @-@ 20th century . In 1953 , an Irish government survey , found that 50 per cent of whiskey drinkers in the United States had never heard of Irish whiskey . \n"
3,
4,
5,"Drought conditions between 1175 and 1180 prompted the Crusaders to sign a two @-@ year truce with the Muslims , but without Tripoli included in the terms . During the 1180s raids by Christians and Muslims into each other 's territory became more frequent . In 1180 , Saladin ventured into the County of Tripoli , ravaging the area . Unwilling to meet him in open battle , the Crusaders retreated to the relative safety of their fortifications . Without capturing the castles , Saladin could not secure control of the area , and once he retreated the Hospitallers were able to revitalize their damaged lands . The Battle of Hattin in 1187 was a disastrous defeat for the Crusaders : Guy of Lusignan , King of Jerusalem , was captured , as was the True Cross , a relic discovered during the First Crusade . Afterwards Saladin ordered the execution of the captured Templar and Hospitaller knights , such was the importance of the two orders in defending the Crusader states . After the battle , the Hospitaller castles of Belmont , Belvoir , and Bethgibelin fell to Muslim armies . Following these losses , the Order focused its attention on its castles in Tripoli . In May 1188 Saladin led an army to attack Krak des Chevaliers , but on seeing the castle decided it was too well defended and instead marched on the Hospitaller castle of Margat , which he also failed to capture . \n"
6,"A number of television programmes have included references to Aston Villa over the past few decades . In the sitcom Porridge , the character Lennie Godber is a Villa supporter . When filming began on Dad 's Army , Villa fan Ian Lavender was allowed to choose Frank Pike 's scarf from an array in the BBC wardrobe ; he chose a claret and blue one — Aston Villa 's colours . The character Nessa in the BBC sitcom Gavin & Stacey was revealed as an Aston Villa fan in an episode screened in December 2009 . In the BBC series "" Yes Minister "" / "" Yes Prime Minister "" , the Minister Jim Hacker 's local team was Aston Villa , as was the Member of Parliament for Birmingham East . \n"
7,"Wrapped in Red 's lead single , "" Underneath the Tree "" , was released to radio airplay on November 5 , 2013 . Praised in its initial release , music critics approvingly compared the song to "" All I Want for Christmas is You "" and blazoned it as a future Christmas standard . Reviewing for Slant Magazine , Sal Cinquemani wrote that track is likely to become Clarkson 's very own contemporary standard ; while The Independent 's Hugh Montgomery applauded it as "" a winner on all fronts . "" After debuting on the Billboard Ho

As we can see, some of the texts are a full paragraph of a Wikipedia article while others are just titles or empty lines.

## Causal Language modeling

For causal language modeling (CLM) we are going to take all the texts in our dataset and concatenate them after they are tokenized. Then we will split them in examples of a certain sequence length. This way the model will receive chunks of contiguous text that may look like:
```
part of text 1
```
or 
```
end of text 1 [BOS_TOKEN] beginning of text 2
```
depending on whether they span over several of the original texts in the dataset or not. The labels will be the same as the inputs, shifted to the left.

We will use the [`distilgpt2`](https://huggingface.co/distilgpt2) model for this example. You can pick any of the checkpoints listed [here](https://huggingface.co/models?filter=causal-lm) instead:

In [11]:
model_checkpoint = "distilgpt2"

To tokenize all our texts with the same vocabulary that was used when training the model, we have to download a pretrained tokenizer. This is all done by the `AutoTokenizer` class:

In [12]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

We can now call the tokenizer on all our texts. This is very simple, using the [`map`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map) method from the Datasets library. First we define a function that call the tokenizer on our texts:

In [13]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [14]:
datasets["train"].column_names

['text']

In [15]:
tokenize_function(datasets["train"][10])

{'input_ids': [383, 983, 705, 82, 3344, 1080, 837, 262, 1086, 72, 51, 57, 1080, 837, 318, 5281, 625, 3264, 422, 569, 18354, 8704, 17740, 764, 5856, 10566, 837, 1938, 2922, 1123, 4326, 1262, 257, 1353, 2488, 12, 31, 866, 6650, 286, 262, 13480, 3975, 1058, 1752, 257, 2095, 318, 6163, 837, 262, 2137, 6100, 262, 2095, 1088, 262, 13480, 287, 2368, 2488, 12, 31, 1048, 764, 317, 2095, 460, 691, 719, 1752, 583, 2488, 12, 31, 1210, 837, 475, 3435, 460, 307, 7520, 3294, 4962, 379, 262, 10907, 286, 584, 3435, 705, 4962, 764, 5501, 2095, 468, 257, 2214, 290, 5253, 286, 3356, 3614, 416, 511, 7561, 35094, 469, 764, 3205, 284, 5193, 3435, 460, 307, 8686, 284, 257, 2060, 4365, 764, 5856, 11327, 837, 3435, 481, 869, 503, 611, 1223, 4325, 284, 606, 837, 884, 355, 511, 1535, 2173, 357, 6574, 1267, 1972, 1877, 393, 852, 13642, 503, 416, 4472, 3434, 764, 5501, 2095, 468, 2176, 366, 6902, 14817, 366, 837, 4678, 3748, 284, 1123, 2095, 764, 1119, 389, 9086, 656, 366, 15644, 32480, 366, 837, 543, 389, 28690, 4

Then we apply it to all the splits in our `datasets` object, using `batched=True` and 4 processes to speed up the preprocessing. We won't need the `text` column afterward, so we discard it.

In [16]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=1, remove_columns=["text"])

Loading cached processed dataset at C:\Users\Admin\.cache\huggingface\datasets\wikitext\wikitext-2-raw-v1\1.0.0\a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126\cache-7e08abd08b7825dd.arrow
Loading cached processed dataset at C:\Users\Admin\.cache\huggingface\datasets\wikitext\wikitext-2-raw-v1\1.0.0\a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126\cache-9216adbba228627f.arrow
Loading cached processed dataset at C:\Users\Admin\.cache\huggingface\datasets\wikitext\wikitext-2-raw-v1\1.0.0\a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126\cache-2bcee6e0178df6b6.arrow


In [17]:
print(type(tokenized_datasets))
tokenized_datasets["train"][0]
tokenized_datasets

<class 'datasets.dataset_dict.DatasetDict'>


DatasetDict({
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 3760
    })
})

If we now look at an element of our datasets, we will see the text have been replaced by the `input_ids` the model will need:

In [18]:
tokenized_datasets["train"][1]

{'input_ids': [796, 569, 18354, 7496, 17740, 6711, 796, 220, 198],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

Now for the harder part: we need to concatenate all our texts together then split the result in small chunks of a certain `block_size`. To do this, we will use the `map` method again, with the option `batched=True`. This option actually lets us change the number of examples in the datasets by returning a different number of examples than we got. This way, we can create our new samples from a batch of examples.

First, we grab the maximum length our model was pretrained with. This might be a big too big to fit in your GPU RAM, so here we take a bit less at just 128.

In [19]:
# block_size = tokenizer.model_max_length
block_size = 128

Then we write the preprocessing function that will group our texts:

In [20]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

First note that we duplicate the inputs for our labels. This is because the model of the 🤗 Transformers library apply the shifting to the right, so we don't need to do it manually.

Also note that by default, the `map` method will send a batch of 1,000 examples to be treated by the preprocessing function. So here, we will drop the remainder to make the concatenated tokenized texts a multiple of `block_size` every 1,000 examples. You can adjust this behavior by passing a higher batch size (which will also be processed slower). You can also speed-up the preprocessing by using multiprocessing:

In [30]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=1,
)

Loading cached processed dataset at C:\Users\Admin\.cache\huggingface\datasets\wikitext\wikitext-2-raw-v1\1.0.0\a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126\cache-5f6dc90dada7f050.arrow
Loading cached processed dataset at C:\Users\Admin\.cache\huggingface\datasets\wikitext\wikitext-2-raw-v1\1.0.0\a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126\cache-0f28ab7d017bfbea.arrow
Loading cached processed dataset at C:\Users\Admin\.cache\huggingface\datasets\wikitext\wikitext-2-raw-v1\1.0.0\a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126\cache-01f106e266a00bf9.arrow


And we can check our datasets have changed: now the samples contain chunks of `block_size` contiguous tokens, potentially spanning over several of our original texts.

In [22]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

' game and follows the " Nameless ", a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven ". \n The game began development in 2010, carrying over a large portion of the work done on Valkyria Chronicles II. While it retained the standard features of the series, it also underwent multiple adjustments, such as making the game more forgiving for series newcomers. Character designer Raita Honjou and composer Hitoshi Sakimoto both returned from previous entries, along with Valkyria Chronicles II director Takeshi Oz'

In [23]:
len(lm_datasets["train"][2]["input_ids"])
print(lm_datasets["train"][2])

{'input_ids': [6909, 764, 317, 1588, 1074, 286, 8786, 12118, 262, 4226, 764, 383, 983, 705, 82, 4756, 7505, 373, 23568, 416, 1737, 705, 77, 764, 220, 198, 632, 1138, 351, 3967, 4200, 287, 2869, 837, 290, 373, 15342, 416, 1111, 4960, 290, 8830, 9188, 764, 2293, 2650, 837, 340, 2722, 41496, 2695, 837, 1863, 351, 281, 9902, 8313, 287, 3389, 286, 326, 614, 764, 632, 373, 635, 16573, 656, 15911, 290, 281, 2656, 2008, 11034, 2168, 764, 14444, 284, 1877, 4200, 286, 569, 18354, 7496, 17740, 2873, 837, 569, 18354, 7496, 17740, 6711, 373, 407, 36618, 837, 475, 257, 4336, 11059, 11670, 351, 262, 983, 705, 82, 9902, 8313, 373, 2716, 287, 1946, 764, 6343, 13, 44206, 561, 1441, 284, 262, 8663, 351, 262, 2478, 286, 569, 18354, 7496], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [31]:
# Load dataset from disk
new_lm_datasets = datasets.load_from_disk("CV_new_lm_datasets")

In [35]:
print(lm_datasets)
print(new_lm_datasets)

DatasetDict({
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2210
    })
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 18667
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1930
    })
})
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 7
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 8
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 7
    })
})


In [37]:
print(lm_datasets["train"][0])
print(new_lm_datasets["train"][0])

{'input_ids': [796, 569, 18354, 7496, 17740, 6711, 796, 220, 198, 2311, 73, 13090, 645, 569, 18354, 7496, 513, 1058, 791, 47398, 17740, 357, 4960, 1058, 10545, 230, 99, 161, 254, 112, 5641, 44444, 9202, 25084, 24440, 12675, 11839, 18, 837, 6578, 764, 569, 18354, 7496, 286, 262, 30193, 513, 1267, 837, 8811, 6412, 284, 355, 569, 18354, 7496, 17740, 6711, 2354, 2869, 837, 318, 257, 16106, 2597, 2488, 12, 31, 2712, 2008, 983, 4166, 416, 29490, 290, 6343, 13, 44206, 329, 262, 14047, 44685, 764, 28728, 287, 3269, 2813, 287, 2869, 837, 340, 318, 262, 2368, 983, 287, 262, 569, 18354, 7496, 2168, 764, 12645, 278, 262, 976, 21748, 286, 16106, 290, 1103, 2488, 12, 31, 640, 11327, 355, 663, 27677, 837, 262, 1621, 4539, 10730, 284, 262, 717], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

Now that the data has been cleaned, we're ready to instantiate our `Trainer`. We will a model:

In [25]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

And some `TrainingArguments`:

In [26]:
from transformers import Trainer, TrainingArguments

In [27]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/gpt-finetuned-wikitext2"` or `"huggingface/gpt-finetuned-wikitext2"`).

We pass along all of those to the `Trainer` class:

In [28]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

And we can train our model:

In [29]:
trainer.train()

c:\Python38\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 7
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3
  Number of trainable parameters = 81912576
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: anforsm (dt2112g1). Use `wandb login --relogin` to force relogin


  0%|          | 0/3 [00:00<?, ?it/s]

RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "c:\Python38\lib\site-packages\torch\nn\parallel\parallel_apply.py", line 64, in _worker
    output = module(*input, **kwargs)
  File "c:\Python38\lib\site-packages\torch\nn\modules\module.py", line 1194, in _call_impl
    return forward_call(*input, **kwargs)
  File "c:\Python38\lib\site-packages\transformers\models\gpt2\modeling_gpt2.py", line 1043, in forward
    transformer_outputs = self.transformer(
  File "c:\Python38\lib\site-packages\torch\nn\modules\module.py", line 1194, in _call_impl
    return forward_call(*input, **kwargs)
  File "c:\Python38\lib\site-packages\transformers\models\gpt2\modeling_gpt2.py", line 887, in forward
    outputs = block(
  File "c:\Python38\lib\site-packages\torch\nn\modules\module.py", line 1194, in _call_impl
    return forward_call(*input, **kwargs)
  File "c:\Python38\lib\site-packages\transformers\models\gpt2\modeling_gpt2.py", line 3

Once the training is completed, we can evaluate our model and get its perplexity on the validation set like this:

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 1930
  Batch size = 16
100%|██████████| 121/121 [00:10<00:00, 11.59it/s]

Perplexity: 38.64


You can now upload the result of the training to the Hub, just execute this instruction: